# Procesamiento del texto, extraemos todo el texto del pdf y lo guardamos en un archivo de texto.

In [ ]:
import fitz  # PyMuPDF
import os

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    
    for page in doc:
        text += page.get_text("text") + "\n"  # Extrae texto de cada página
    
    return text.strip()

# Ruta de tu PDF
pdf_path = "../data/raw/leyes_ajedrez.pdf"  # Cambia esto por la ubicación de tu archivo

# Extraer texto
texto_extraido = extract_text_from_pdf(pdf_path)

# Crear carpeta si no existe    
output_dir = "../data/cooked"
os.makedirs(output_dir, exist_ok=True)

# Guardar el texto en un archivo dentro de 'data/cooked'
output_path = os.path.join(output_dir, "texto_extraido.txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(texto_extraido)

print(f"✅ Extracción completada. El texto se guardó en '{output_path}'.")

✅ Extracción completada. El texto se guardó en '../data/cooked\texto_extraido.txt'.


# La base de datos elegida, ha sido ChromaDB

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar el modelo preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')  # Puedes cambiar el modelo si lo deseas

# Leer el archivo de texto
with open('../data/cooked/texto_extraido.txt', 'r') as f:
    lines = f.readlines()

# Generar embeddings para cada línea del archivo
embeddings = model.encode(lines, convert_to_numpy=True)

# Ver el primer embedding para verificar
print("Embedding de la primera línea:", embeddings[0])

c:\Users\almaz\.conda\envs\prueba_rag_tailor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\almaz\.conda\envs\prueba_rag_tailor\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\almaz\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admin

Embedding de la primera línea: [-4.75044623e-02  4.89124469e-02  1.27341738e-02  3.71110216e-02
 -6.25089034e-02  6.24664910e-02  1.16436899e-01  1.00796316e-02
  4.63477075e-02  3.58318053e-02  2.38445606e-02  1.86951656e-03
 -4.92336135e-03  1.57713201e-02  3.41440667e-03 -7.20255524e-02
 -2.60634460e-02  6.74121603e-02 -6.22904599e-02  6.52326504e-03
  6.79132789e-02 -2.33350545e-02 -4.56841663e-03  1.08755054e-02
 -9.58510786e-02 -3.89817655e-02  5.08660404e-03  1.80085637e-02
 -3.48556116e-02 -7.94901252e-02  8.54951441e-02 -6.16623648e-03
  3.67506035e-02 -4.58656140e-02 -6.46752566e-02 -4.30934355e-02
  5.96923865e-02 -3.52040194e-02  5.32939211e-02  5.61981611e-02
 -1.11851528e-01  6.75243279e-03 -9.78955925e-02 -4.36949432e-02
 -2.68482175e-02 -5.19135632e-02  5.89504763e-02 -4.86038029e-02
  1.68348681e-02  5.84590202e-03 -3.57686505e-02 -6.13384880e-02
  2.84125417e-04  3.52543741e-02  4.63330820e-02  8.03012308e-03
  3.54612730e-02  2.48460993e-02  3.17950733e-02  1.1308147

In [4]:
import faiss

# Dimensión de los embeddings (depende del modelo que uses, por ejemplo, 384 para 'all-MiniLM-L6-v2')
dim = embeddings.shape[1]

# Crear el índice FAISS (usamos IndexFlatL2 para distancia euclidiana)
index = faiss.IndexFlatL2(dim)

# Agregar los embeddings al índice
index.add(embeddings)

# Verifica cuántos vectores se han añadido
print("Número de vectores en el índice:", index.ntotal)

Número de vectores en el índice: 1827


In [8]:
consulta = ["cuando se gana una partida"]
consulta_embedding = model.encode(consulta, convert_to_numpy=True)

In [9]:
# Buscar los 5 vectores más cercanos
k = 5
distances, indices = index.search(consulta_embedding, k)

print("Índices de los vectores más cercanos:", indices)
print("Distancias de los vectores más cercanos:", distances)

Índices de los vectores más cercanos: [[483 760 937 655 370]]
Distancias de los vectores más cercanos: [[0.5913608  0.5915155  0.595267   0.61422104 0.6148168 ]]


In [10]:
# Guardar el índice
faiss.write_index(index, "mi_indice.index")

# Cargar el índice desde el archivo
index_cargado = faiss.read_index("mi_indice.index")

In [11]:
import faiss

# Número de dimensiones de los embeddings (debería coincidir con la salida de tu modelo)
dim = embeddings.shape[1]

# Crear el índice FAISS (puedes elegir el tipo de índice dependiendo de tus necesidades)
index = faiss.IndexFlatL2(dim)  # IndexFlatL2 es para distancias L2

# Agregar los embeddings al índice
index.add(embeddings)  # embeddings es un array numpy de tamaño (número de textos, dimensiones del embedding)
print(f"Se han agregado {embeddings.shape[0]} embeddings al índice.")

Se han agregado 1827 embeddings al índice.


In [15]:
# Función para realizar la búsqueda en FAISS
def buscar_similaridad(query, k=5):
    # Generar el embedding para la consulta
    query_embedding = model.encode([query], convert_to_numpy=True)
    
    # Realizar la búsqueda en FAISS
    D, I = index.search(query_embedding, k)  # D es la distancia, I los índices
    
    # Mostrar los resultados
    for i in range(k):
        print(f"Documento recuperado: {lines[I[0][i]]}\nDistancia: {D[0][i]}")

# Prueba de búsqueda con una consulta
query = "¿Qué es tablas?"
buscar_similaridad(query)

Documento recuperado: Sin embargo, la partida es tablas si la posición es tal que el oponente no puede dar 

Distancia: 0.4856340289115906
Documento recuperado: la consideración de una oferta de tablas. 

Distancia: 0.5014404058456421
Documento recuperado: La partida es tablas cuando se alcanza una posición en la que ningún jugador puede 

Distancia: 0.5126779079437256
Documento recuperado: Si la posición es tal que ninguno de los jugadores puede dar mate, la partida es tablas. (Ver 

Distancia: 0.5383651852607727
Documento recuperado: embargo, decretará tablas si la posición es tal que el oponente no puede dar jaque 

Distancia: 0.5409550070762634


In [ ]:
import openai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import time
import os

# Cargar las variables de entorno
from dotenv import load_dotenv
load_dotenv()

# Obtener la clave de API de OpenAI desde las variables de entorno
openai.api_key = os.getenv('OPENAI_API_KEY')

# Cargar el modelo preentrenado para embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Cargar el índice FAISS previamente guardado
index = faiss.read_index("mi_indice.index")

# Leer los documentos originales desde el archivo
with open('../data/cooked/texto_extraido.txt', 'r') as f:
    lines = f.readlines()

# Función para buscar documentos relevantes en FAISS
def buscar_documentos(query, k=5):
    # Obtener el embedding para la consulta utilizando el modelo
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Buscar en el índice de FAISS los documentos más cercanos
    D, I = index.search(query_embedding, k)

    # Recuperar los textos de los documentos más cercanos
    documents = [lines[i] for i in I[0]]
    return documents

# Función para crear un hilo de conversación en OpenAI
def create_thread(openai_client):
    """
    Crea un nuevo hilo de conversación en OpenAI.
    """
    return openai_client.beta.threads.create()

# Función para enviar un mensaje a OpenAI y obtener la respuesta
def process_data(openai_client, assistant_id, thread_id, message):
    """
    Envía un mensaje a un asistente de OpenAI y procesa su respuesta.
    """
    openai_client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message,
    )

    run = openai_client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    run_status = openai_client.beta.threads.runs.retrieve(
        thread_id=thread_id,
        run_id=run.id
    )

    while True:
        run_status = openai_client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )
        if run_status.status == "completed":
            break
        elif run_status.status == "failed":
            print("Error, no se encontró una respuesta del asistente.")
            return "No se encontró una respuesta del asistente."
        else:
            time.sleep(3)

    response_messages = openai_client.beta.threads.messages.list(thread_id=thread_id)
    
    assistant_response = None
    for message in response_messages.data:
            assistant_response = "\n".join([block.text.value for block in message.content])
            break

    return assistant_response

# Función para generar una respuesta utilizando los documentos recuperados
def generar_respuesta(query):
    # Obtener documentos relevantes de FAISS
    documents = buscar_documentos(query)
    
    # Crear el contexto para el asistente, concatenando los documentos relevantes
    contexto = "\n".join(documents)

    # Inicializa el cliente de OpenAI
    openai_client = openai.OpenAI(api_key=openai.api_key)

    # Crear el hilo
    thread = create_thread(openai_client)
    thread_id = thread['id']

    # Enviar la consulta al asistente
    assistant_response = process_data(openai_client, assistant_id="assistant_id", thread_id=thread_id, message=f"Basado en los siguientes documentos, por favor, responde a la consulta: '{query}'\n\n{contexto}")

    return assistant_response

# Ejemplo de uso
query = "¿Qué dice el documento sobre inteligencia artificial?"
respuesta = generar_respuesta(query)
print("Respuesta generada:", respuesta)


ModuleNotFoundError: No module named 'streamlit'

In [24]:
# Ejemplo de consulta
consulta = "¿Cuál es el propósito del sistema de búsqueda?"

# Generar respuesta usando el sistema
respuesta = generar_respuesta(consulta)

print("Respuesta generada:", respuesta)

AttributeError: module 'openai' has no attribute 'Chat'